In [302]:
import os
import json
import pandas as pd
import traceback

In [303]:
from langchain.chat_models import ChatOpenAI


In [304]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [305]:
KEY=os.getenv("OPENAI_API_KEY")

In [306]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [307]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [308]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [309]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [310]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [311]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [312]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [313]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [314]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [315]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [316]:
file_path=r"/Users/user/Downloads/data.txt"

In [317]:
file_path

'/Users/user/Downloads/data.txt'

In [318]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [319]:
print(TEXT)

Tornadoes are violently rotating columns of air, extending from a thunderstorm, which are in contact with the ground. Tornadoes develop when wind variations with height support rotation in the updraft. As seen below, tornadoes come in different sizes, many as narrow rope-like swirls, others as wide funnels.
Across the Plains, tornadoes can be seen from miles away. However, in the southeast, and especially Georgia, tornadoes are often hidden in large swaths of rain and hail, making them very difficult to see and even more dangerous. Visibility is often affected by terrain constraints in Georgia as well.
As stated before, tornadoes come in different shapes and sizes. They are ranked using the Enhanced Fujita scale. The majority of tornadoes which occur are classified as a weak tornado. Usually a weak tornado will last for just a few minutes and have wind speeds of 100 mph or less. Some tornadoes intensify further and become strong or violent. Strong tornadoes last for twenty minutes or m

In [320]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [321]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [322]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Tornadoes are violently rotating columns of air, extending from a thunderstorm, which are in contact with the ground. Tornadoes develop when wind variations with height support rotation in the updraft. As seen below, tornadoes come in different sizes, many as narrow rope-like swirls, others as wide funnels.
Across the Plains, tornadoes can be seen from miles away. However, in the southeast, and especially Georgia, tornadoes are often hidden in large swaths of rain and hail, making them very difficult to see and even more dangerous. Visibility is often affected by terrain constraints in Georgia as well.
As stated before, tornadoes come in different shapes and sizes. They are ranked using the Enhanced Fujita scale. The majority of tornadoes which occur are classified as a weak tornado. Usually a weak tornado will last for just a few minutes and have wind speeds of 100 mph or less. 

In [323]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1316
Prompt Tokens:938
Completion Tokens:378
Total Cost:0.002163


In [324]:
response

{'text': "Tornadoes are violently rotating columns of air, extending from a thunderstorm, which are in contact with the ground. Tornadoes develop when wind variations with height support rotation in the updraft. As seen below, tornadoes come in different sizes, many as narrow rope-like swirls, others as wide funnels.\nAcross the Plains, tornadoes can be seen from miles away. However, in the southeast, and especially Georgia, tornadoes are often hidden in large swaths of rain and hail, making them very difficult to see and even more dangerous. Visibility is often affected by terrain constraints in Georgia as well.\nAs stated before, tornadoes come in different shapes and sizes. They are ranked using the Enhanced Fujita scale. The majority of tornadoes which occur are classified as a weak tornado. Usually a weak tornado will last for just a few minutes and have wind speeds of 100 mph or less. Some tornadoes intensify further and become strong or violent. Strong tornadoes last for twenty 

In [325]:
quiz=response.get("quiz")

In [326]:
quiz=json.loads(quiz)

In [327]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [328]:
quiz_table_data

[{'MCQ': 'What are tornadoes?',
  'Choices': 'a: Violently rotating columns of air extending from a thunderstorm | b: Gentle breezes in the sky | c: Giant waves in the ocean | d: Heavy rain showers',
  'Correct': 'a'},
 {'MCQ': 'How do tornadoes develop?',
  'Choices': 'a: When there is no wind | b: When wind variations with height support rotation in the updraft | c: When the sun is shining bright | d: When there is a snowstorm',
  'Correct': 'b'},
 {'MCQ': 'Where are tornadoes often hidden in Georgia?',
  'Choices': 'a: In large swaths of rain and hail | b: On the beach | c: In the desert | d: In the mountains',
  'Correct': 'a'},
 {'MCQ': 'How are tornadoes classified?',
  'Choices': 'a: Using the Richter scale | b: Using the Enhanced Fujita scale | c: Using the Celsius scale | d: Using the Beaufort scale',
  'Correct': 'b'},
 {'MCQ': 'What is the wind speed of a weak tornado?',
  'Choices': 'a: 50 mph | b: 100 mph or less | c: 150 mph | d: 200 mph',
  'Correct': 'b'}]

In [329]:
quiz=pd.DataFrame(quiz_table_data)

In [330]:
quiz.to_csv("machinelearning.csv",index=False)

In [331]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_31_2024_16_55_51'